In [1]:
from PIL import Image

import sklearn
from sklearn import cluster

import skimage
from skimage import color

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random
import os

import tools

import scipy
from scipy import spatial

In [2]:
input_datapath = './PlantVillage/'

output_datapath = './Processed/'

potato_healthy = input_datapath + 'Potato___healthy/'
potato_late_blight = input_datapath + 'Potato___Late_blight/'
potato_early_blight = input_datapath + 'Potato___Early_blight/'

potato_healthy_output = output_datapath + 'Potato___healthy/'
potato_late_blight_output = output_datapath + 'Potato___Late_blight/'
potato_early_blight_output = output_datapath + 'Potato___Early_blight/'

actual_tater = "actual_tater.jpg"

In [3]:
def segment_leaf(image: Image.Image, resolution: tuple = (50, 50)):

    original_size = image.size
    original_image_array = np.array(image)

    # reducing resolution reduces compute
    image = image.resize(resolution)

    # convert to L*A*B for better representation
    image_array_rgb = np.array(image)
    image_array_lab = color.rgb2lab(image_array_rgb)

    # fit model
    model = cluster.KMeans(n_clusters=8)
    lab_flattened = image_array_lab.reshape(-1, 3)
    lab_classified = model.fit_predict(lab_flattened)

    # boolean array representing pixels with/out suffecient green, 1 if L > 10 and A < -10, else 0
    binary = np.array([1 if model.cluster_centers_[i][1] < -10 and model.cluster_centers_[i][0] > 10 else 0 for i in lab_classified]).reshape(image_array_lab.shape[:2])

    # array of all coordinates in binary grid that meet leaf threshold
    leaf_coordinates = np.array([[j,binary.shape[1] - i] for i in range(binary.shape[0]) for j in range(binary.shape[1]) if binary[i,j] == 1])

    # convex hull
    hull = spatial.ConvexHull(leaf_coordinates)

    # boolean array, 1 if inside convex hull
    samples = np.array([[i,j] for i in range(binary.shape[0]) for j in range(binary.shape[1])])
    eps = 1e-9
    inside = (hull.equations@np.c_[samples, np.ones(len(samples))].T < -eps).all(0)
    full_hull = samples[inside]
    for each in full_hull:
        binary[binary.shape[0]-each[1]][each[0]] = 1

    # map boolean array onto original image
    binary_colors = np.array([[0,0,0], [1,1,1]])
    rgb_binary = np.array(Image.fromarray(np.uint8(binary_colors[binary].reshape(image_array_lab.shape))).resize(original_size))
    segmented_image = Image.fromarray(np.uint8(rgb_binary * original_image_array))

    return segmented_image


In [4]:
images = tools.load_image_sample(potato_healthy, as_array=False)

i = 0
for image in images:
    segment_leaf(image, resolution=(128, 128)).save(potato_healthy_output+str(i)+".jpg")
    i += 1

KeyboardInterrupt: 

In [ ]:
images = tools.load_image_sample(potato_early_blight, as_array=False)

i = 0
for image in images:
    try:
        segment_leaf(image,resolution=(128, 128)).save(potato_early_blight_output+str(i)+".jpg")
        i += 1
    except:
        print("image segmentation failed")

In [ ]:
images = tools.load_image_sample(potato_late_blight, as_array=False)

i = 0
for image in images:
    try:
        segment_leaf(image, resolution=(128, 128)).save(potato_late_blight_output+str(i)+".jpg")
        i += 1
    except:
        print("image segmentation failed")